# Scenario 2 - Randomized counts for Tableau
### For the Digital Health Research Infrastructure Specialist Position

**Jessa Green  |  jessa@jessagreen.com  |  (920) 850-4950  |  [My LinkedIn Profile](https://www.linkedin.com/in/jessagreen/)**

Below is the process I followed to create a fictional dataset based on numbers I was able to gather from your public information. This dataset was used to populate the tables presented in the Tableau dashboard provided as the solution to Scenario 2 in the interview process.

> Note: These are not real data or numbers for The PRIDE Study. Everything shown here is a fictional scenario based a few real-world numbers I gathered.

In [1]:
# Import needed libraries/packages
import pandas as pd
import numpy as np
import random
from datetime import datetime
import calendar

In [2]:
# Create variables based on publicly known PRIDE Studies numbers

# Participant mid-year counts from medical journal
ps_mid18 = 10952
ps_mid19 = 13731
# Fictional participant counts created
ps_mid20 = 16436
ps_mid21 = 19853
# Participant count shown in my PS account dashboard on 2022-01-21
ps22 = 23952


# The PRIDE Study response percentage for the 2017 AQ
aq17 = 0.658
# The PRIDE Study response percentage for the 2018 AQ (11 month window)
aq18pt = 0.479
# Create a 12-month percentage based on a basic average
aq18 = (aq18pt / 11) * 12
print("This gives us a 'full year' response rate of {} to use for 2018 AQ".format(round(aq18, 3)))

This gives us a 'full year' response rate of 0.523 to use for 2018 AQ


In [3]:
# Create fictional response rates for each year post 2018

aq19 = np.random.uniform((aq18-0.12), (aq17+0.12))
aq20 = np.random.uniform((aq18-0.12), (aq17+0.12))
aq21 = np.random.uniform((aq18-0.12), (aq17+0.12))

print("This gives us response rates of: \n2019 AQ: {:.3f} \n2020 AQ: {:.3f} \n2021 AQ: {:.3f}".format(aq19, aq20, aq21))


This gives us response rates of: 
2019 AQ: 0.730 
2020 AQ: 0.464 
2021 AQ: 0.748


### Create function to generate a set of monthly counts
The below function will generate a set of 12 random counts that will total a given amount when summed.

In [4]:
# Now create fictional monthly response counts based on annual numbers

def gen_monthly_counts(n, total):
# Return a randomly chosen list of n positive integers summing to total.
    dividers = sorted(random.sample(range(1, total), n - 1))
    return [a - b for a, b in zip(dividers + [total], [0] + dividers)]


In [5]:
# Create dictionary of years and counts to create our dataset for Tableau
ps_dict = {2018: ps_mid18, 2019: ps_mid19, 2020: ps_mid20, 2021: ps_mid21}


### Create the dataset contents
Here we generate the entire lists consisting of each of the years, the months, and the participation counts using the monthly randomizer function so that our annual counts match our summed monthly counts for each year.

In [6]:
month_counts = []
month_years = []
just_years = []

for key,value in ps_dict.items():
    month_counts.extend(gen_monthly_counts(12, value))
    for i in range(12):      
        new_date = datetime.strptime((str(i+1) + '/' + str(key)),'%m/%Y').date()
        month_years.append(new_date)
        just_years.append(key)


In [7]:
df = pd.DataFrame(list(zip(just_years, month_years, month_counts)))
df.columns =['Yr', 'Date', 'Participants']

In [8]:
df.head(12)

,Yr,Date,Participants
0,2018,2018-01-01,1264
1,2018,2018-02-01,2560
2,2018,2018-03-01,1998
3,2018,2018-04-01,99
4,2018,2018-05-01,837
5,2018,2018-06-01,102
6,2018,2018-07-01,844
7,2018,2018-08-01,2
8,2018,2018-09-01,2055
9,2018,2018-10-01,603


### Randomized monthly counts fit annual counts

The below comparison confitms that my randomized dataset has created random monthly counts for participation to be 
used in our visualized data:

In [9]:
print(df.query('Yr == 2018').Participants.sum(), '<-- Sum of randomized monthly counts for 2018')
print(ps_dict[2018], '<-- Count from the created annual variables at the top')

10952 <-- Sum of randomized monthly counts for 2018
10952 <-- Count from the created annual variables at the top


In [10]:
print(df.query('Yr == 2021').Participants.sum(), '<-- Sum of randomized monthly counts for 2021')
print(ps_dict[2021], '<-- Count from the created annual variables at the top')

19853 <-- Sum of randomized monthly counts for 2021
19853 <-- Count from the created annual variables at the top


In [11]:
df.drop('Yr', axis=1, inplace=True)
df.head()

,Date,Participants
0,2018-01-01,1264
1,2018-02-01,2560
2,2018-03-01,1998
3,2018-04-01,99
4,2018-05-01,837


### Convert Random Counts to Actual Responses
Finally, I take the monthly counts and assign them each an individual "response" by state to align with the number of responses each month/year randomized above.

In [12]:
test = dict(zip(df.Date, df.Participants))
states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [13]:
all_part_states_list = []
all_part_dates_list = []

for i,j in test.items():
    for count in range(j):
        random.choice(states)
        all_part_states_list.append(random.choice(states))
        all_part_dates_list.append(i)


In [15]:
df2 = pd.DataFrame()
df2['month_yr'] = all_part_dates_list
df2['state'] = all_part_states_list

In [16]:
df2.head() 

,month_yr,state
0,2018-01-01,OK
1,2018-01-01,PA
2,2018-01-01,VA
3,2018-01-01,ME
4,2018-01-01,AZ


In [17]:
df2.shape

(60972, 2)

### Export the dataset to CSV to use in Tableau

In [18]:
df2.to_csv('pride_study_participation.csv', index=False)


*Known participant numbers gathered from Journal of the American Medical Informatics Association, 2019, Vol. 26, No.8-9, pg.745-746